In [2]:
import numpy as np
import pandas as pd

In [3]:
recipes = pd.read_csv('../data/RAW_recipes.csv')

In [4]:
recipes

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,1 brownies in the world best ever,333281,40,985201,2008-10-27,"['60-minutes-or-less', 'time-to-make', 'course...","[138.4, 10.0, 50.0, 3.0, 3.0, 19.0, 6.0]",10,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9
1,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,"['60-minutes-or-less', 'time-to-make', 'cuisin...","[595.1, 46.0, 211.0, 22.0, 13.0, 51.0, 26.0]",12,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11
2,412 broccoli casserole,306168,40,50969,2008-05-30,"['60-minutes-or-less', 'time-to-make', 'course...","[194.8, 20.0, 6.0, 32.0, 22.0, 36.0, 3.0]",6,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9
3,millionaire pound cake,286009,120,461724,2008-02-12,"['time-to-make', 'course', 'cuisine', 'prepara...","[878.3, 63.0, 326.0, 13.0, 20.0, 123.0, 39.0]",7,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",7
4,2000 meatloaf,475785,90,2202916,2012-03-06,"['time-to-make', 'course', 'main-ingredient', ...","[267.0, 30.0, 12.0, 12.0, 29.0, 48.0, 2.0]",17,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",13
...,...,...,...,...,...,...,...,...,...,...,...,...
83777,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
83778,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
83779,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
83780,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


**Data Cleaning**

*Removing Missingness*

In [29]:
#Finding columns with missing vals
nan_columns = recipes.columns[recipes.isnull().any()]

print(nan_columns)

Index(['name', 'description'], dtype='object')


In [41]:
recipes[recipes['name'].isna()] # 1 Missing
recipes[recipes['description'].isna()] # 70 Missing

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
1486,almond cookie bites,401761,16,1434673,2009-11-29,"['30-minutes-or-less', 'time-to-make', 'course...","[40.1, 3.0, 4.0, 1.0, 0.0, 8.0, 1.0]",16,"['preheat oven to 350 degrees f', 'in medium b...",NaN,"['all-purpose flour', ""fisher chef's naturals ...",9
3087,apricot gorgonzola crescent appetizers,332410,40,991676,2008-10-22,"['60-minutes-or-less', 'time-to-make', 'course...","[139.9, 11.0, 15.0, 7.0, 6.0, 13.0, 4.0]",17,['heat oven to 350f spray large cookie sheet w...,NaN,['pillsbury refrigerated crescent dinner rolls...,6
3685,asparagus milanese,382664,15,714468,2009-07-24,"['15-minutes-or-less', 'time-to-make', 'course...","[225.2, 26.0, 13.0, 7.0, 23.0, 45.0, 3.0]",16,"['snap off the tough ends of the asparagus', '...",NaN,"['asparagus', 'parmigiano-reggiano cheese', 'b...",5
5803,baked yams with spicy molasses butter,382792,75,714468,2009-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[482.9, 24.0, 27.0, 5.0, 8.0, 49.0, 27.0]",10,"['using electric mixer , beat first 7 ingredie...",NaN,"['unsalted butter', 'molasses', 'ground cinnam...",8
5983,balthazar s brioche french toast,378728,25,714468,2009-06-24,"['30-minutes-or-less', 'time-to-make', 'course...","[836.3, 39.0, 84.0, 42.0, 63.0, 67.0, 40.0]",14,"['in a large bowl , whisk together eggs , suga...",NaN,"['eggs', 'superfine sugar', 'milk', 'ground ci...",10
...,...,...,...,...,...,...,...,...,...,...,...,...
79329,ultimate screwdriver,423077,2,1596512,2010-05-04,"['15-minutes-or-less', 'time-to-make', 'course...","[40.6, 0.0, 30.0, 0.0, 1.0, 0.0, 3.0]",3,"['fill glass with ice', 'add smirnoff orange f...",NaN,"['smirnoff orange vodka', 'orange juice']",2
80877,voodoo chicken with sweet and sour onions ro...,383510,30,714468,2009-07-31,"['30-minutes-or-less', 'time-to-make', 'course...","[263.4, 7.0, 33.0, 3.0, 53.0, 4.0, 3.0]",8,['preheat oven to 400f coat a rimmed baking sh...,NaN,"['boneless skinless chicken breast', 'salt and...",8
81188,wasatch mountain chili,290480,50,778477,2008-03-06,"['60-minutes-or-less', 'time-to-make', 'course...","[672.9, 42.0, 25.0, 54.0, 65.0, 33.0, 25.0]",4,['in a large saucepan over medium heat cook on...,NaN,"['onion', 'olive oil', 'hominy', 'great northe...",14
81701,white bean chicken chili giada de laurentiis,430591,75,714468,2010-06-22,"['time-to-make', 'course', 'main-ingredient', ...","[409.0, 21.0, 8.0, 35.0, 87.0, 18.0, 10.0]",13,['in a large heavy-bottomed saucepan or dutch ...,NaN,"['olive oil', 'onion', 'garlic cloves', 'groun...",18


In [5]:
recipes.iloc[238]['steps']


"['in a bowl , combine ingredients except for olive oil', 'slowly whisk inches', 'olive oil until thickened', 'great with field greens', 'makes about 2 / 3', 'cup dressing']"

In [6]:
recipes['name'].iloc[238] = 'Salad Vinegarette'

/var/folders/fj/nnphscc51ydgzq5pylw9n6w80000gn/T/ipykernel_41587/823503736.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  recipes['name'].iloc[238] = 'Salad Vinegarette'
/var/folders/fj/nnphscc51ydgzq5pylw9n6w80000gn/T/ipykernel_41587/8

In [9]:
# Fill missing descriptions with a placeholder
recipes['description'] = recipes['description'].fillna('No description available')

print(f"Number of missing descriptions after filling: {recipes['description'].isna().sum()}")


Number of missing descriptions after filling: 0


In [10]:
nan_columns = recipes.columns[recipes.isnull().any()]

print(nan_columns)

Index([], dtype='object')


Exploding Nutritional Information

In [11]:
# Convert nutrition column from string to list
recipes['nutrition'] = recipes['nutrition'].apply(eval)


In [21]:
# Convert nutrition list into separate columns
nutrition_df = pd.DataFrame(recipes['nutrition'].tolist(), 
                          columns=['calories', 'total fat (PDV)', 'sugar (PDV)', 'sodium (PDV)', 'protein (PDV)', 'saturated fat (PDV)', 'carbohydrates (PDV)'])

# Add the nutrition columns to the original dataframe
recipes2 = pd.concat([recipes, nutrition_df], axis=1)


In [22]:
recipes2


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,1 brownies in the world best ever,333281,40,985201,2008-10-27,"['60-minutes-or-less', 'time-to-make', 'course...","[138.4, 10.0, 50.0, 3.0, 3.0, 19.0, 6.0]",10,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9,138.4,10.0,50.0,3.0,3.0,19.0,6.0
1,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,"['60-minutes-or-less', 'time-to-make', 'cuisin...","[595.1, 46.0, 211.0, 22.0, 13.0, 51.0, 26.0]",12,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11,595.1,46.0,211.0,22.0,13.0,51.0,26.0
2,412 broccoli casserole,306168,40,50969,2008-05-30,"['60-minutes-or-less', 'time-to-make', 'course...","[194.8, 20.0, 6.0, 32.0, 22.0, 36.0, 3.0]",6,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9,194.8,20.0,6.0,32.0,22.0,36.0,3.0
3,millionaire pound cake,286009,120,461724,2008-02-12,"['time-to-make', 'course', 'cuisine', 'prepara...","[878.3, 63.0, 326.0, 13.0, 20.0, 123.0, 39.0]",7,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",7,878.3,63.0,326.0,13.0,20.0,123.0,39.0
4,2000 meatloaf,475785,90,2202916,2012-03-06,"['time-to-make', 'course', 'main-ingredient', ...","[267.0, 30.0, 12.0, 12.0, 29.0, 48.0, 2.0]",17,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",13,267.0,30.0,12.0,12.0,29.0,48.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83777,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,415.2,26.0,34.0,26.0,44.0,21.0,15.0
83778,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,14.8,0.0,2.0,58.0,1.0,0.0,1.0
83779,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,59.2,6.0,2.0,3.0,6.0,5.0,0.0
83780,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,188.0,11.0,57.0,11.0,7.0,21.0,9.0


Converting Times to DateTime

In [16]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])

Merging Recipes and Interactions (TODO)